In [1]:
%load_ext lab_black

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import geopandas as gpd
import pandas as pd
import numpy as np
import lxml

from urbana.constants import DIR_REPO, DIR_DATA

In [4]:
# papermill parameters cell
# https://papermill.readthedocs.io/en/latest/usage-parameterize.html

YEAR = 2017
MONTH = 7

SAVE_INTERIM = True
OUTPUT_WARNINGS = False

In [5]:
if not OUTPUT_WARNINGS:
    import warnings

    warnings.filterwarnings("ignore")

# Open Data



## City maps

We will take a list of the neighbourhoods and districts with information to plot them from:

https://opendata-ajuntament.barcelona.cat/data/en/dataset/20170706-districtes-barris

In [6]:
geo_info = gpd.read_file(
    DIR_DATA / "raw/open_data/0301100100_UNITATS_ADM_POLIGONS.json"
)

# Districts
dist = geo_info[geo_info["SCONJ_DESC"] == "Districte"][["DISTRICTE", "NOM", "geometry"]]
dist = dist[["DISTRICTE", "NOM", "geometry"]]
dist.rename({"DISTRICTE": "N_district", "NOM": "District"}, axis=1, inplace=True)
dist["N_district"] = dist["N_district"].astype("int")
dist["District"] = (
    dist["District"]
    .str.lower()
    .str.replace("-", " ")
    .str.replace(",", " ")
    .str.replace("  ", " ")
    .str.replace("  ", " ")
    .str.replace("  ", " ")
    .str.normalize("NFKD")
    .str.encode("ascii", errors="ignore")
    .str.decode("utf-8")
)
dist.set_index("N_district", inplace=True)

# Neighbourhoods
neigh = geo_info[geo_info["SCONJ_DESC"] == "Barri"]
neigh = neigh[["DISTRICTE", "BARRI", "NOM", "geometry"]]
neigh.rename(
    {"BARRI": "N_neighbourhood", "NOM": "Neighbourhood", "DISTRICTE": "N_district"},
    axis=1,
    inplace=True,
)
neigh["N_neighbourhood"] = neigh["N_neighbourhood"].astype("int")
neigh["N_district"] = neigh["N_district"].astype("int")
neigh["Neighbourhood"] = (
    neigh["Neighbourhood"]
    .str.lower()
    .str.replace("-", " ")
    .str.replace(",", " ")
    .str.replace("  ", " ")
    .str.replace("  ", " ")
    .str.replace("  ", " ")
    .str.normalize("NFKD")
    .str.encode("ascii", errors="ignore")
    .str.decode("utf-8")
)
neigh.set_index("N_neighbourhood", inplace=True)
neigh = pd.merge(
    left=neigh,
    right=dist[["District"]],
    how="inner",
    left_on="N_district",
    right_index=True,
)

We save the information required to plot the maps (`neigh_geo`) in a JSON file. From now on, we will store the rest of the information in `neigh` so it can be stores in a csv file that is easier to read for humans.

In [7]:
neigh_geo = neigh["geometry"]
neigh.drop("geometry", axis=1, inplace=True)
if SAVE_INTERIM:
    neigh_geo.to_file(DIR_DATA / "interim/neigh_geo.json", driver="GeoJSON")

## Mean income

We can extract the mean income **per census section** from:

https://opendata-ajuntament.barcelona.cat/data/en/dataset/renda-tributaria-per-persona-atlas-distribucio

In order to pass from sections to neighbourhoods, we need the population of each section.
This information can be found at the INE:

https://www.ine.es/dyngs/INEbase/en/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990

Once we choose a year and a city (in this case, 2007 and Barcelona), we will get to:

https://www.ine.es/jaxi/tabla.do?path=/t20/e245/p07/a2017/l1/&file=0801.px&type=pcaxis&L=1

Here we have to choose the sections by entering their codes. If we want all the sections of the city of Barcelona, we have to select all the codes that star by *08019*, since *08* is for the province of Barcelona and 019 is for the municipality.

The two following numbers will be the district (from 0 to 10) and the last three numbers are the census section.

For example, the code 0801905013 means: the province of Barcelona (08), then the municipality of Barcelona (019), then the neighbourhood Sarrià-Sant Gervasi (05), take the section number 013.

In [8]:
census = pd.read_csv(
    DIR_DATA / "raw/open_data/2017_census.csv",
    sep=";",
    thousands=".",
    encoding="latin-1",
)
census["N_district"] = census["Sección"].astype("str").str[4:6].astype("int")
census["N_section"] = census["Sección"].astype("str").str[6:9].astype("int")
census.drop(["Sección", "Sexo", "Edad (grupos quinquenales)"], axis=1, inplace=True)
census.rename({"Total": "Population"}, axis=1, inplace=True)

income = pd.read_csv(
    DIR_DATA / "raw/open_data/2017_rendatributariamitjanaperpersona.csv"
)
income.drop(["Any", "Nom_Districte", "Nom_Barri"], axis=1, inplace=True)
income.rename(
    {
        "Codi_Districte": "N_district",
        "Codi_Barri": "N_neighbourhood",
        "Seccio_Censal": "N_section",
    },
    axis=1,
    inplace=True,
)

income = pd.merge(
    left=income,
    right=census,
    how="inner",
    left_on=["N_district", "N_section"],
    right_on=["N_district", "N_section"],
)

list_sections = income[["N_district", "N_neighbourhood", "N_section", "Population"]]


income["Import_Euros_Population"] = income["Import_Euros"].multiply(
    income["Population"]
)
income = (
    income.groupby(["N_district", "N_neighbourhood"])[
        "Population", "Import_Euros_Population"
    ]
    .agg("sum")
    .reset_index()
)
income["Mean_Income_Year"] = income["Import_Euros_Population"].div(income["Population"])
# income['Mean_Income_Month'] = income['Mean_Income_Year']/12
income.drop(["Population", "Import_Euros_Population"], axis=1, inplace=True)
income.set_index("N_neighbourhood", inplace=True)
income

neigh = pd.merge(
    left=neigh,
    right=income[["Mean_Income_Year"]],
    how="inner",
    left_index=True,
    right_index=True,
)

## Age and gender

We can extract information regarding the age and gender of the population by neighbourhood from:

https://opendata-ajuntament.barcelona.cat/data/en/dataset/est-ine-edat-any-a-any

We will take 2017, since the income data also comes from that year.

We are also going to store the total population by neighbourhood in a variable called "total_neigh", which will be useful later to calculate different percentages.

In [9]:
age_gender = pd.read_csv(
    DIR_DATA / "raw/open_data/2017_ine_edat_any_a_any_per_sexe.csv"
)
age_gender.drop(
    ["Any", "Codi_Districte", "Nom_Districte", "Nom_Barri"], axis=1, inplace=True
)
age_gender.rename(
    {
        "Codi_Barri": "N_neighbourhood",
        "Sexe": "Gender",
        "Edat any a any": "Age",
        "Nombre": "Number",
    },
    axis=1,
    inplace=True,
)
age_gender["Gender"].replace(["Home", "Dona"], ["Male", "Female"], inplace=True)
age_gender["Age"] = age_gender["Age"].str[:2].astype(int)

total_neigh = age_gender.groupby(["N_neighbourhood"])["Number"].sum()
total_neigh

N_neighbourhood
1     47608
2     16062
3     14996
4     22721
5     32016
      ...  
69    13629
70    23009
71    20487
72    26146
73    28691
Name: Number, Length: 73, dtype: int64

In [10]:
gender_neigh = (
    age_gender.groupby(["N_neighbourhood", "Gender"])["Number"]
    .sum()
    .reset_index()
    .set_index("N_neighbourhood")
)
gender_neigh = gender_neigh[gender_neigh["Gender"] == "Female"]
gender_neigh["Population_Female"] = gender_neigh["Number"].div(total_neigh)

neigh = pd.merge(
    left=neigh,
    right=gender_neigh[["Population_Female"]],
    how="inner",
    left_index=True,
    right_index=True,
)

We define the following population groups:


*   From 0 to 15.
*   From 16 to 25.
*   From 26 to 65.
*   Over 65



In [11]:
age_groups = (
    age_gender.groupby(
        ["N_neighbourhood", pd.cut(age_gender["Age"], [-1, 15, 25, 65, 99])]
    )["Number"]
    .sum()
    .reset_index()
)
age_groups["Age"] = age_groups["Age"].astype("str")
# Group 1 -> [0, 15]
group1 = (
    age_groups[age_groups["Age"] == "(-1, 15]"]
    .set_index("N_neighbourhood")
    .drop("Age", axis=1)
)
group1["Population_0_15"] = group1["Number"].div(total_neigh)
# Group 2 -> [16, 25]
group2 = (
    age_groups[age_groups["Age"] == "(15, 25]"]
    .set_index("N_neighbourhood")
    .drop("Age", axis=1)
)
group2["Population_16_25"] = group2["Number"].div(total_neigh)
# Group 3 -> [26, 65]
group3 = (
    age_groups[age_groups["Age"] == "(25, 65]"]
    .set_index("N_neighbourhood")
    .drop("Age", axis=1)
)
group3["Population_26_65"] = group3["Number"].div(total_neigh)
# Group 4 -> [66, 99]
group4 = (
    age_groups[age_groups["Age"] == "(65, 99]"]
    .set_index("N_neighbourhood")
    .drop("Age", axis=1)
)
group4["Population_66_Plus"] = group4["Number"].div(total_neigh)

neigh = pd.merge(
    left=neigh,
    right=group1[["Population_0_15"]],
    how="inner",
    left_index=True,
    right_index=True,
)
neigh = pd.merge(
    left=neigh,
    right=group2[["Population_16_25"]],
    how="inner",
    left_index=True,
    right_index=True,
)
neigh = pd.merge(
    left=neigh,
    right=group3[["Population_26_65"]],
    how="inner",
    left_index=True,
    right_index=True,
)
neigh = pd.merge(
    left=neigh,
    right=group4[["Population_66_Plus"]],
    how="inner",
    left_index=True,
    right_index=True,
)

## Economic activities

We can extract information about the economic activities of many businesses of Barcelona from:

https://opendata-ajuntament.barcelona.cat/data/en/dataset/cens-activitats-comercials

And in order to know interpret the codes, we will use:

https://opendata-ajuntament.barcelona.cat/data/en/dataset/cens-activitats-economiques-class-bcn

In [12]:
econ = pd.read_csv(DIR_DATA / "raw/open_data/2019_censcomercialbcn_detall.csv")

econ = econ[
    [
        "Codi_Sector_Activitat",
        "Codi_Grup_Activitat",
        "Codi_Activitat_2019",
        "Nom_Grup_Activitat",
        "Codi_Barri",
    ]
]
econ.rename(
    {
        "Codi_Sector_Activitat": "Code_sector",
        "Codi_Grup_Activitat": "Code_group",
        "Nom_Grup_Activitat": "Name_Group",
        "Codi_Activitat_2019": "Code_activity",
        "Codi_Barri": "N_neighbourhood",
    },
    axis=1,
    inplace=True,
)

# Group 0 means 'no information' so we remove it
econ = econ[econ["Code_group"] != 0]

# # Group by codes and count
econ = (
    econ.groupby(
        ["N_neighbourhood", "Code_sector", "Code_group", "Code_activity", "Name_Group"]
    )
    .size()
    .to_frame("Count")
    .reset_index()
)
econ

,N_neighbourhood,Code_sector,Code_group,Code_activity,Name_Group,Count
0,1,1,1,1000010,Quotidià alimentari,3
1,1,1,1,1000020,Quotidià alimentari,102
2,1,1,1,1000030,Quotidià alimentari,75
3,1,1,1,1001000,Quotidià alimentari,7
4,1,1,1,1002000,Quotidià alimentari,70
...,...,...,...,...,...,...
4141,73,3,17,1700320,Altres,3
4142,73,3,17,1700400,Altres,6
4143,73,3,17,1700500,Altres,93
4144,73,3,17,1700600,Altres,50


In [13]:
def sum_Points(acts):
    result = pd.DataFrame(index=neigh.index)
    empty = pd.DataFrame(index=neigh.index)
    for i in acts:
        empty[str(i)] = econ[econ["Code_activity"] == i].set_index("N_neighbourhood")[
            "Count"
        ]
    empty.fillna(0, inplace=True)
    result["myResult"] = empty.sum(axis=1)
    return result["myResult"]

In [14]:
econ_list = pd.DataFrame(index=neigh.index)
econ_group = (
    econ.groupby(["N_neighbourhood", "Name_Group"])["Count"]
    .agg("sum")
    .to_frame("Count")
    .reset_index()
)

# LIST OF CATEGORIES DEFINED
econ_list["POI_Daily_Food"] = (
    econ_group[econ_group["Name_Group"] == "Quotidià alimentari"]
    .drop("Name_Group", axis=1)
    .set_index("N_neighbourhood")
)
econ_list["POI_Daily_Others"] = (
    econ_group[econ_group["Name_Group"] == "Quotidià no alimentari"]
    .drop("Name_Group", axis=1)
    .set_index("N_neighbourhood")
)
econ_list["POI_House_Euipment"] = (
    econ_group[econ_group["Name_Group"] == "Parament de la llar"]
    .drop("Name_Group", axis=1)
    .set_index("N_neighbourhood")
)
econ_list["POI_Clothing"] = (
    econ_group[econ_group["Name_Group"] == "Equipament personal"]
    .drop("Name_Group", axis=1)
    .set_index("N_neighbourhood")
)
econ_list["POI_Culture"] = (
    econ_group[econ_group["Name_Group"] == "Oci i cultura"]
    .drop("Name_Group", axis=1)
    .set_index("N_neighbourhood")
)
econ_list["POI_Vehicles"] = (
    econ_group[econ_group["Name_Group"] == "Automoció"]
    .drop("Name_Group", axis=1)
    .set_index("N_neighbourhood")
)
econ_list["POI_State_Agents"] = (
    econ_group[econ_group["Name_Group"] == "Activitats immobiliàries"]
    .drop("Name_Group", axis=1)
    .set_index("N_neighbourhood")
)
econ_list["POI_Education"] = (
    econ_group[econ_group["Name_Group"] == "Ensenyament"]
    .drop("Name_Group", axis=1)
    .set_index("N_neighbourhood")
)
econ_list["POI_Finances"] = (
    econ_group[econ_group["Name_Group"] == "Finances i assegurances"]
    .drop("Name_Group", axis=1)
    .set_index("N_neighbourhood")
)
econ_list["POI_Restaurants_Hotels"] = (
    econ_group[
        econ_group["Name_Group"]
        == "Restaurants, bars i hotels (Inclòs hostals, pensions i fondes)"
    ]
    .drop("Name_Group", axis=1)
    .set_index("N_neighbourhood")
)
econ_list["POI_Health"] = (
    econ_group[econ_group["Name_Group"] == "Sanitat i assistència"]
    .drop("Name_Group", axis=1)
    .set_index("N_neighbourhood")
)

econ_list["POI_Departments_Stores"] = sum_Points([7000000])
econ_list["POI_Souvenirs_Thrift_Store"] = sum_Points([7004000, 7004010, 7004020])
econ_list["POI_Car_Parks"] = sum_Points([1600000])
econ_list["POI_Esthetics"] = sum_Points([1600600, 1600700, 1600800])
econ_list["POI_Sports"] = sum_Points([1700300, 1700310, 1700320])
econ_list["POI_Religion"] = sum_Points([1700400])
econ_list["POI_Administration"] = sum_Points([1700100])


econ_list.fillna(0, inplace=True)
econ_list = econ_list.astype("int")

# Merge the data
neigh = pd.merge(
    left=neigh, right=econ_list, how="inner", left_index=True, right_index=True
)

# Inside Airbnb

We can extract information about the Airbnbs in Barcelona from:

http://insideairbnb.com/get-the-data.html

Instead of dowloading the files by clicking them one by one, we can exploit the fact that they all have the same url structure to download them straight from pandas. This is useful in the case in ehich we would like to download them all.

'http://data.insideairbnb.com/spain/catalonia/barcelona/' + *Date in "%Y-%m-%d" format* + '/data/listings.csv.gz'

## Available data

In [15]:
# Dates in which the files were uploaded (so far)
airbnb_dates = [
    "2015-04-30",
    "2015-07-17",
    "2015-09-04",
    "2015-10-02",
    "2016-01-03",
    "2016-11-07",
    "2016-12-08",
    "2017-01-04",
    "2017-02-09",
    "2017-03-06",
    "2017-04-08",
    "2017-05-07",
    "2017-06-05",
    "2017-07-06",
    "2017-08-06",
    "2017-09-12",
    "2017-10-07",
    "2017-11-13",
    "2017-12-09",
    "2018-01-17",
    "2018-02-07",
    "2018-04-12",
    "2018-05-14",
    "2018-06-09",
    "2018-07-10",
    "2018-08-14",
    "2018-09-11",
    "2018-10-10",
    "2018-11-07",
    "2018-12-10",
    "2019-01-14",
    "2019-02-06",
    "2019-03-08",
    "2019-04-10",
    "2019-05-14",
    "2019-06-07",
    "2019-07-10",
    "2019-08-12",
    "2019-09-17",
    "2019-10-16",
    "2019-11-09",
    "2019-12-10",
    "2020-01-10",
    "2020-02-16",
    "2020-03-16",
    "2020-04-16",
    "2020-05-11",
    "2020-06-13",
    "2020-07-17",
    "2020-08-24",
    "2020-09-12",
    "2020-10-12",
    "2020-11-06",
    "2020-12-16",
    "2021-01-12",
    "2021-02-09",
]

# Choose the data to download
airbnb_chosen = [
    i for i in airbnb_dates if i.startswith(str(YEAR) + "-" + "{:02d}".format(MONTH))
]
airbnb_chosen = pd.to_datetime(airbnb_chosen[0], format="%Y-%m-%d", errors="ignore")

## Process data

In [16]:
def df_standarize(df):
    df.index = (
        df.index.str.lower()
        .str.replace("-", " ")
        .str.replace(",", " ")
        .str.replace("  ", " ")
        .str.replace("  ", " ")
        .str.replace("  ", " ")
        .str.normalize("NFKD")
        .str.encode("ascii", errors="ignore")
        .str.decode("utf-8")
    )
    return df

In [17]:
airbnb_number_neigh = neigh["Neighbourhood"]
airbnb_price_neigh = neigh["Neighbourhood"]
airbnb_price_person_neigh = neigh["Neighbourhood"]
airbnb_loc_review_neigh = neigh["Neighbourhood"]

###########################################################
airbnb_url = (
    "http://data.insideairbnb.com/spain/catalonia/barcelona/"
    + airbnb_chosen.strftime("%Y-%m-%d")
    + "/data/listings.csv.gz"
)
df = pd.read_csv(airbnb_url)


date = airbnb_chosen.strftime("%Y_%m")

# Some rows are wrong
df = df[df["price"].str.contains("-") == False]

# Make the prices integers and remove outliers (prices over 1000)
df["price"] = df["price"].str.replace("$", "").str.replace(",", "").astype(float)
df = df[df["price"] < 1000]
df["price_person"] = df["price"].div(df["accommodates"])

# Extract the data from the file
temp_number_neigh = (
    df["neighbourhood_cleansed"].value_counts().rename("Airbnb_Number").sort_index()
)
temp_price_neigh = (
    df.groupby(["neighbourhood_cleansed"])["price"]
    .agg("mean")
    .rename("Airbnb_Price")
    .sort_index()
)
temp_price_person_neigh = (
    df.groupby(["neighbourhood_cleansed"])["price_person"]
    .agg("mean")
    .rename("Airbnb_Price_Person")
    .sort_index()
)
temp_loc_review_neigh = (
    df.groupby(["neighbourhood_cleansed"])["review_scores_location"]
    .agg("mean")
    .rename("Airbnb_Location_Score")
    .sort_index()
)


# Standarize the names
df_standarize(temp_number_neigh)
df_standarize(temp_price_neigh)
df_standarize(temp_price_person_neigh)
df_standarize(temp_loc_review_neigh)

# Merge the new data with the pre-existing
airbnb_number_neigh = pd.merge(
    left=airbnb_number_neigh,
    right=temp_number_neigh,
    how="left",
    left_on="Neighbourhood",
    right_index=True,
)
airbnb_price_neigh = pd.merge(
    left=airbnb_price_neigh,
    right=temp_price_neigh,
    how="left",
    left_on="Neighbourhood",
    right_index=True,
)
airbnb_price_person_neigh = pd.merge(
    left=airbnb_price_person_neigh,
    right=temp_price_person_neigh,
    how="left",
    left_on="Neighbourhood",
    right_index=True,
)
airbnb_loc_review_neigh = pd.merge(
    left=airbnb_loc_review_neigh,
    right=temp_loc_review_neigh,
    how="left",
    left_on="Neighbourhood",
    right_index=True,
)


###########################################################3
# Remove useless information
airbnb_number_neigh.drop(["Neighbourhood"], axis=1, inplace=True)
airbnb_price_neigh.drop(["Neighbourhood"], axis=1, inplace=True)
airbnb_price_person_neigh.drop(["Neighbourhood"], axis=1, inplace=True)
airbnb_loc_review_neigh.drop(["Neighbourhood"], axis=1, inplace=True)


# Add all the data together
neigh = pd.merge(
    left=neigh,
    right=airbnb_number_neigh,
    how="inner",
    left_index=True,
    right_index=True,
)
neigh = pd.merge(
    left=neigh, right=airbnb_price_neigh, how="inner", left_index=True, right_index=True
)
neigh = pd.merge(
    left=neigh,
    right=airbnb_price_person_neigh,
    how="inner",
    left_index=True,
    right_index=True,
)
neigh = pd.merge(
    left=neigh,
    right=airbnb_loc_review_neigh,
    how="inner",
    left_index=True,
    right_index=True,
)

# Barcelona City Council

  We are once again going to take data from 2017, beacause in the year for which we have more infomration available.

## People living alone

We can extract information about the amount of people living alone by neighbourhood from:

http://www.bcn.cat/estadistica/angles/dades/tpob/pad/padro/a2017/solas/sola05.htm

In [18]:
table_alone = pd.read_html(
    "https://www.bcn.cat/estadistica/castella/dades/tpob/pad/padro/a"
    + str(YEAR)
    + "/solas/sola05.htm",
    thousands=".",
)
alone = table_alone[0]
alone.columns = alone.iloc[4]
alone.drop(labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 82, 83, 84, 85, 86], axis=0, inplace=True)
alone = alone.loc[:, ~alone.columns.duplicated()]
alone[["N_district", "Name_neigh"]] = alone["Dto. Barrios"].str.split(
    " ", 1, expand=True
)
alone[["N_neighbourhood", "Name"]] = alone["Name_neigh"].str.split(". ", 1, expand=True)
alone.columns.name = None
alone["N_neighbourhood"] = alone["N_neighbourhood"].astype("int")
alone.set_index("N_neighbourhood", inplace=True)
alone.drop(
    ["Dto. Barrios", "TOTAL", "Name", "Name_neigh", "N_district"], axis=1, inplace=True
)
alone = alone.astype("int")
alone["Alone_25_64"] = alone.iloc[:, 3:11].sum(axis=1)
alone["Alone_65_Plus"] = alone.iloc[:, 11:19].sum(axis=1)
alone.rename({"< 25 años": "Alone_18_25"}, axis=1, inplace=True)
alone.drop(alone.columns[1:17], axis=1, inplace=True)

for i in alone.columns:
    alone[i] = alone[i].div(total_neigh)

neigh = pd.merge(
    left=neigh, right=alone, how="inner", left_index=True, right_index=True
)

## Mean household

Data coming from:

http://www.bcn.cat/estadistica/angles/dades/tpob/pad/padro/a2017/llars/ocu02.htm

In [19]:
table_household = pd.read_html(
    "https://www.bcn.cat/estadistica/castella/dades/tpob/pad/padro/a"
    + str(YEAR)
    + "/llars/ocu02.htm",
    thousands=".",
    decimal=",",
)
household = table_household[0]
household.columns = household.iloc[4]
household.drop(labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 82, 83, 84, 85], axis=0, inplace=True)
household = household.loc[:, ~household.columns.duplicated()]
household[["N_district", "Name_neigh"]] = household["Dto. Barrios"].str.split(
    " ", 1, expand=True
)
household[["N_neighbourhood", "Name"]] = household["Name_neigh"].str.split(
    ". ", 1, expand=True
)
household.columns.name = None
household["N_neighbourhood"] = household["N_neighbourhood"].astype("int")
household.set_index("N_neighbourhood", inplace=True)
household.rename(
    {"Ocupación media (personas por domicilio)": "Household"}, axis=1, inplace=True
)
household = household[["Household"]]
household = household.astype("float")

neigh = pd.merge(
    left=neigh, right=household, how="inner", left_index=True, right_index=True
)

## Educational level

Given the Spanish education system, the groups are defined as:


*   Elementary School = Primaria (6-12 years)
*   Middle School = Secundaria (13-16 years)
*   High School = Bachillerato (17-18 years)

The data is extracted from:


http://www.bcn.cat/estadistica/angles/dades/tpob/pad/padro/a2017/nivi/nivi05.htm

In [20]:
table_education = pd.read_html(
    "https://www.bcn.cat/estadistica/castella/dades/tpob/pad/padro/a"
    + str(YEAR)
    + "/nivi/nivi05.htm",
    thousands=".",
)
education = table_education[0]
education.columns = education.iloc[4]
education.drop(
    labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 82, 83, 84, 85, 86, 87], axis=0, inplace=True
)
education = education.loc[:, ~education.columns.duplicated()]
education[["N_district", "Name_neigh"]] = education["Dto. Barrios"].str.split(
    " ", 1, expand=True
)
education[["N_neighbourhood", "Name"]] = education["Name_neigh"].str.split(
    ". ", 1, expand=True
)
education.columns.name = None
education["N_neighbourhood"] = education["N_neighbourhood"].astype("int")
education.set_index("N_neighbourhood", inplace=True)
education.drop(
    ["Dto. Barrios", "TOTAL", "Name", "Name_neigh", "N_district", "No consta"],
    axis=1,
    inplace=True,
)
education = education.astype("int")
education.rename(
    {
        "Sin estudios": "Education_None",
        "Estudios primarios / certificado de escolaridad / EGB": "Education_Primary_School",
        "Bachillerato elemental / graduado escolar / ESO / FPI": "Education_Middle_School",
        "Bachillerato superior / BUP / COU / FPII / CFGM grado medio": "Education_High_School",
        "Estudios universitarios / CFGS grado superior": "Education_University",
    },
    axis=1,
    inplace=True,
)

for i in education.columns:
    education[i] = education[i].div(total_neigh)


neigh = pd.merge(
    left=neigh, right=education, how="inner", left_index=True, right_index=True
)

## Country of origin

The data can be found at:

http://www.bcn.cat/estadistica/angles/dades/tpob/pad/padro/a2017/nacio/nacio05.htm

In [21]:
table_nationality = pd.read_html(
    "https://www.bcn.cat/estadistica/castella/dades/tpob/pad/padro/a"
    + str(YEAR)
    + "/nacio/nacio05.htm",
    thousands=".",
)
nationality = table_nationality[0]
nationality.columns = nationality.iloc[5]
nationality.drop(
    labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 83, 84, 85, 86], axis=0, inplace=True
)
nationality = nationality.loc[:, ~nationality.columns.duplicated()]
nationality[["N_district", "Name_neigh"]] = nationality["Dto. Barrios"].str.split(
    " ", 1, expand=True
)
nationality[["N_neighbourhood", "Name"]] = nationality["Name_neigh"].str.split(
    ". ", 1, expand=True
)
nationality.columns.name = None
nationality["N_neighbourhood"] = nationality["N_neighbourhood"].astype("int")
nationality.set_index("N_neighbourhood", inplace=True)
nationality.drop(
    [
        "Dto. Barrios",
        "TOTAL",
        "Total",
        "Resto países",
        "Apátridas / No consta",
        "Name",
        "Name_neigh",
        "N_district",
    ],
    axis=1,
    inplace=True,
)
nationality.dropna(axis=1, inplace=True)
nationality = nationality.astype("int")
nationality.rename(
    {
        "España": "Nationality_Spain",
        "Italia": "Nationality_Italy",
        "Francia": "Nationality_France",
        "Reino Unido": "Nationality_UK",
        "Rumanía": "Nationality_Romania",
        "Alemania": "Nationality_Germany",
        "Rusia": "Nationality_Russia",
        "Ucrania": "Nationality_Ucrany",
        "Portugal": "Nationality_Portugal",
        "Países Bajos": "Nationality_Netherlands",
        "Polonia": "Nationality_Poland",
        "Bulgaria": "Nationality_Bulgaria",
        "Suecia": "Nationality_Sweden",
        "Bélgica": "Nationality_Belgium",
        "Irlanda": "Nationality_Ireland",
        "Grecia": "Nationality_Greece",
        "Marruecos": "Nationality_Morocco",
        "Argelia": "Nationality_Argelia",
        "Senegal": "Nationality_Senegal",
        "Bolivia": "Nationality_Bolivia",
        "Colombia": "Nationality_Colombia",
        "Perú": "Nationality_Peru",
        "Ecuador": "Nationality_Ecuator",
        "Honduras": "Nationality_Honduras",
        "Brasil": "Nationality_Brazil",
        "Venezuela": "Nationality_Venezuela",
        "Argentina": "Nationality_Argentina",
        "República Dominicana": "Nationality_Dominican_Republic",
        "Estados Unidos, los": "Nationality_USA",
        "México": "Nationality_Mexico",
        "Paraguay": "Nationality_Paraguay",
        "Chile": "Nationality_Chile",
        "Cuba": "Nationality_Cuba",
        "Uruguay": "Nationality_Uruguay",
        "el Salvador": "Nationality_Salvador",
        "China": "Nationality_China",
        "Pakistán": "Nationality_Pakistan",
        "Filipinas": "Nationality_Philippines",
        "India": "Nationality_India",
        "Bangladesh": "Nationality_Bangladesh",
        "Georgia": "Nationality_Georgia",
        "Armenia": "Nationality_Armenia",
        "Japón": "Nationality_Japan",
        "Nepal": "Nationality_Nepal",
        "Oceanía": "Nationality_Oceania",
    },
    axis=1,
    inplace=True,
)

for i in nationality.columns:
    nationality[i] = nationality[i].div(total_neigh)

neigh = pd.merge(
    left=neigh, right=nationality, how="inner", left_index=True, right_index=True
)

# Generalitat de Catalunya

http://habitatge.gencat.cat/ca/dades/estadistiques_publicacions/indicadors_estadistiques/estadistiques_de_construccio_i_mercat_immobiliari/mercat_de_lloguer/lloguers-barcelona-per-districtes-i-barris/index.html

## Number of renting contracts

In [22]:
rent_number = pd.read_excel(
    "http://habitatge.gencat.cat/web/.content/home/dades/estadistiques/01_Estadistiques_de_construccio_i_mercat_immobiliari/03_Mercat_de_lloguer/03_Lloguers_Barcelona_per_districtes_i_barris/anual_bcn_contractes.xls",
    usecols=range(10),
    skiprows=18,
    skipfooter=3,
)

rent_number.columns = [
    "N_neighbourhood",
    "Neighbouhood",
    "2020",
    "2019",
    "2018",
    "2017",
    "2016",
    "2015",
    "2014",
    "2013",
]

rent_number.set_index("N_neighbourhood", inplace=True)

rent_number = rent_number[[str(YEAR)]]

rent_number.rename({str(YEAR): "Rent_Number"}, axis=1, inplace=True)

rent_number["Rent_Number"].replace("n.d.", np.nan, inplace=True)

neigh = pd.merge(
    left=neigh, right=rent_number, how="inner", left_index=True, right_index=True
)

## Renting Price

In [23]:
rent_price = pd.read_excel(
    "http://habitatge.gencat.cat/web/.content/home/dades/estadistiques/01_Estadistiques_de_construccio_i_mercat_immobiliari/03_Mercat_de_lloguer/03_Lloguers_Barcelona_per_districtes_i_barris/anual_bcn_lloguer.xls",
    usecols=range(10),
    skiprows=18,
    skipfooter=3,
)

rent_price.columns = [
    "N_neighbourhood",
    "Neighbouhood",
    "2020",
    "2019",
    "2018",
    "2017",
    "2016",
    "2015",
    "2014",
    "2013",
]

rent_price.set_index("N_neighbourhood", inplace=True)

rent_price = rent_price[[str(YEAR)]]

rent_price.rename({str(YEAR): "Rent_Price"}, axis=1, inplace=True)

rent_price["Rent_Price"].replace("n.d.", np.nan, inplace=True)

neigh = pd.merge(
    left=neigh, right=rent_price, how="inner", left_index=True, right_index=True
)

## Renting Price per Area

In [24]:
rent_price_area = pd.read_excel(
    "http://habitatge.gencat.cat/web/.content/home/dades/estadistiques/01_Estadistiques_de_construccio_i_mercat_immobiliari/03_Mercat_de_lloguer/03_Lloguers_Barcelona_per_districtes_i_barris/anual_bcn_lloguer_m2.xls",
    usecols=range(10),
    skiprows=18,
    skipfooter=3,
)

rent_price_area.columns = [
    "N_neighbourhood",
    "Neighbouhood",
    "2020",
    "2019",
    "2018",
    "2017",
    "2016",
    "2015",
    "2014",
    "2013",
]

rent_price_area.set_index("N_neighbourhood", inplace=True)

rent_price_area = rent_price_area[[str(YEAR)]]

rent_price_area.rename({str(YEAR): "Rent_Price_Area"}, axis=1, inplace=True)

rent_price_area["Rent_Price_Area"].replace("n.d.", np.nan, inplace=True)

neigh = pd.merge(
    left=neigh, right=rent_price_area, how="inner", left_index=True, right_index=True
)

## Renting Area

In [25]:
rent_area = pd.read_excel(
    "http://habitatge.gencat.cat/web/.content/home/dades/estadistiques/01_Estadistiques_de_construccio_i_mercat_immobiliari/03_Mercat_de_lloguer/03_Lloguers_Barcelona_per_districtes_i_barris/anual_bcn_sup.xls",
    usecols=range(10),
    skiprows=18,
    skipfooter=3,
)

rent_area.columns = [
    "N_neighbourhood",
    "Neighbouhood",
    "2020",
    "2019",
    "2018",
    "2017",
    "2016",
    "2015",
    "2014",
    "2013",
]

rent_area.set_index("N_neighbourhood", inplace=True)

rent_area = rent_area[[str(YEAR)]]

rent_area.rename({str(YEAR): "Rent_Area"}, axis=1, inplace=True)

rent_area["Rent_Area"].replace("n.d.", np.nan, inplace=True)

neigh = pd.merge(
    left=neigh, right=rent_area, how="inner", left_index=True, right_index=True
)

# Save data

In [26]:
if SAVE_INTERIM:
    neigh.to_csv(DIR_DATA / "interim/neigh.csv")

In [27]:
neigh

,N_district,Neighbourhood,District,Mean_Income_Year,Population_Female,Population_0_15,Population_16_25,Population_26_65,Population_66_Plus,POI_Daily_Food,...,Nationality_Bangladesh,Nationality_Georgia,Nationality_Armenia,Nationality_Japan,Nationality_Nepal,Nationality_Oceania,Rent_Number,Rent_Price,Rent_Price_Area,Rent_Area
N_neighbourhood,,,,,,,,,,,,,,,,,,,,,
1,1,el raval,ciutat vella,9001.176987,0.458032,0.136511,0.112985,0.637792,0.112712,484,...,0.059717,0.001302,0.000924,0.000840,0.003088,0.000693,2003,748.579486,13.915648,59.218221
2,1,el barri gotic,ciutat vella,11978.842548,0.479890,0.089155,0.115365,0.665235,0.130245,106,...,0.006226,0.001806,0.000560,0.002490,0.000872,0.002117,808,973.270161,14.191885,77.628242
3,1,la barceloneta,ciutat vella,11548.549280,0.502867,0.089891,0.101160,0.631635,0.177314,101,...,0.001000,0.000600,0.000533,0.000734,0.001400,0.002134,857,745.814714,19.844592,40.104113
4,1,sant pere santa caterina i la ribera,ciutat vella,12657.310286,0.502267,0.103473,0.104353,0.655561,0.136614,210,...,0.001981,0.002025,0.000352,0.003037,0.000264,0.002069,1309,916.652697,15.713611,66.137255
5,2,el fort pienc,eixample,16376.465455,0.529798,0.122314,0.092454,0.581928,0.203305,136,...,0.001093,0.001000,0.000312,0.000781,0.000718,0.000281,1054,900.418197,13.114796,74.004119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,10,diagonal mar i el front maritim del poblenou,sant marti,17323.097293,0.508475,0.205077,0.065962,0.588304,0.140656,36,...,0.000000,0.000660,0.000660,0.000660,0.000000,0.000367,394,1092.062970,16.031175,94.374172
70,10,el besos i el maresme,sant marti,9605.617715,0.495241,0.153201,0.103481,0.570386,0.172932,106,...,0.001043,0.002564,0.001173,0.000087,0.002651,0.000043,354,613.031243,10.031687,62.358974
71,10,provencals del poblenou,sant marti,14038.842876,0.517645,0.148631,0.090887,0.591204,0.169278,34,...,0.000098,0.001757,0.001025,0.000049,0.000537,0.000049,427,932.015902,12.935797,72.045213


# Watermark

In [28]:
%load_ext watermark

In [29]:
# NBVAL_IGNORE_OUTPUT
%watermark -a "Ernesto Gregori" -u -d -t -v -m -g -wb

Author: Ernesto Gregori

Last updated: 2021-03-18 23:14:17

Python implementation: CPython
Python version       : 3.9.2
IPython version      : 7.21.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.8.0-44-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Git hash: 7977714aae72cdab7e742b07cf8caa0f9e847d2d

Git branch: master

Watermark: 2.2.0



In [30]:
# NBVAL_IGNORE_OUTPUT
%watermark -iv

lxml     : 4.6.2
numpy    : 1.20.1
pandas   : 1.2.3
geopandas: 0.9.0

